<a href="https://colab.research.google.com/github/anetakahle/earthquakes/blob/main/earthquakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Úvod a cíle

Tato zpráva prezentuje analýzu datového souboru zemětřesení v období od roku 1990 do roku 2023. Zemětřesení představují významné přírodní nebezpečí a porozumění jejich vzorcům a charakteristikám je klíčové pro hodnocení a zmírňování rizik. Tato analýza si klade za cíl:

*   Prozkoumat distribuci zemětřesení v čase a prostoru.
*   Identifikovat regiony s nejvyšší frekvencí a magnitudou zemětřesení.
*   Prozkoumat vztah mezi různými charakteristikami a magnitudou zemětřesení.
*   Vytvořit jednoduchý prediktivní model pro výskyt zemětřesení.

# 2. Získávání a příprava dat

## 2.1. Import knihoven


In [ ]:
!pip install umap-learn pandas matplotlib datashader bokeh holoviews scikit-image colorcet
!pip install densmap-learn
!pip install streamlit
!pip install ydata_profiling
!pip install --upgrade plotly
import plotly.express as px
import sklearn.datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import umap
import umap.plot
import streamlit as st
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
import math
import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeRegressor
import matplotlib
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
!pip install country_converter --upgrade
import country_converter as coco
import matplotlib.ticker as mticker
from scipy.stats import linregress
import matplotlib.style as style
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from scipy.stats import f_oneway
warnings.filterwarnings('ignore')
%matplotlib inline
import geopandas as gpd
import nltk
import re
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # Import accuracy_score from sklearn.metrics
from IPython.display import FileLink
from datetime import datetime, timezone
import timeit
from geopy.distance import geodesic as GD
try:
    import haversine as hvsn
    from haversine import Unit
except ModuleNotFoundError:
    !pip install haversine
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
# Use 'dark_background' style instead of 'seaborn-dark-palette'
plt.style.use('dark_background')
# The 'dark_background' style provides a dark theme.
# If you specifically want a Seaborn-like dark theme, you can use:
# plt.style.use('seaborn-darkgrid')
import missingno as mno
import shap
from ydata_profiling import ProfileReport
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.3/260.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20

## 2.2. Stažení a načtení dat

Následující kód stáhne datový soubor a načte ho do Pandas DataFrame.

In [ ]:
# Načtení dat
df = pd.read_csv('Eartquakes-1990-2023.csv')

## 2.3. Kontrola dat

Prozkoumejme prvních několik řádků datového souboru, abychom pochopili jeho strukturu a obsah.

In [ ]:
print("\nPrvních 5 řádků:")
df.head()

In [ ]:
# Základní informace o datasetu
df.info()

In [ ]:
df

In [ ]:
df.describe() # Základní statistiky pro číselné sloupce


In [ ]:
df.isna().sum()

## 2.4. Čištění dat

kontrola duplicitních řádků a jejich smazání

In [ ]:
# Nalezení duplicitních řádků
duplicate_rows = df[df.duplicated(keep='first')]

# Počet duplicitních řádků
num_duplicates = duplicate_rows.shape[0]

# Zobrazení duplicitních řádků
print(f"Počet duplicitních řádků: {num_duplicates}")
duplicate_rows


In [ ]:
# Odstranění duplicitních řádků
df.drop_duplicates(keep='first', inplace=True)

# 3. Explorativní analýza dat (EDA)

## 3.1. Analýza časových řad

Převod sloupce 'date' na objekty datetime a extrakce časové charakteristiky.

In [ ]:
if not pd.api.types.is_datetime64_any_dtype(df['date']):
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time

### 3.1.1. Trend magnitudy podle roku

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
d = df.set_index("date")

d['year'] = d.index.year
x = d['year'].values
y = d['magnitudo'].values

coefficients = np.polyfit(x, y, 1)
linear_fit = np.poly1d(coefficients)

plt.figure(figsize=(16, 5))
plt.plot(d.index, d["magnitudo"], ".", label="Data")
plt.plot(d.index, linear_fit(x), "-", color="red", label="Linear Regression")
plt.title("Trend magnitudy podle roku")
plt.xlabel("Rok")
plt.ylabel("Magnitudo")
plt.legend()
plt.show()


### 3.1.2. Distribuce magnitudy

In [ ]:
plt.figure(figsize = (16,7))
sns.histplot(df["magnitudo"], bins = 250)
plt.title("Bimodální distribuce magnitudy")
plt.show()


## 3.2. Geografická analýza

### 3.2.1. Mapa hustoty zemětřesení

In [ ]:
def filter_plot_day(df, date_1,date_2):
    """ Vrátí mapu filtrovanou podle data"""
    df.copy()
    df = df.loc[(df.date > date_1) & (df.date < date_2)]
    fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='magnitudo', radius=10,
    center=dict(lat=0, lon=180), zoom=0,
    title =f"<br> Magnituda zemětřesení mezi {date_1} a {date_2}")
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()

filter_plot_day(df, date_1 = "2023-02-01", date_2 ="2023-02-02")


### 3.2.2. Nejsilnější zemětřesení podle roku

In [ ]:
def plot_strongest(df):
    # Create 'year' column from 'date' column
    df['year'] = df['date'].dt.year

    gb_top = df.groupby(["state"])\
    .agg({"significance":"max", "magnitudo":"max"})\
    .reset_index()

    b = df[["year", "state","significance","longitude", "latitude","magnitudo"]]
    merge = pd.merge(gb_top, b, how = "left", on =["state","significance", "magnitudo"]).sort_values(by="year")


    fig = px.scatter_geo(merge,
    lat="latitude", lon="longitude",
    size="significance",
    animation_frame="year",
    projection="natural earth",
    title = "Nejsilnější zemětřesení podle roku")
    fig.show()

plot_strongest(df)

## 3.3. Analýza kategorických proměnných

### 3.3.1. Státy s nejvyšším počtem zemětřesení

In [ ]:
# Získání 5 států s nejvyšším počtem zemětřesení
top_5_states = (
    df.groupby("state")
    .size()
    .to_frame(name="count")
    .reset_index()
    .sort_values(by=["count"], ascending=False)
    .head(5)["state"]
)

# Získání unikátních hodnot 5 nejlepších států
top_5_states_unique = top_5_states.unique()

# Tisk unikátních hodnot
print(top_5_states_unique)

# Vytvoření DataFrame s počty zemětřesení
earthquake_counts_df = pd.DataFrame(
    data={"state": top_5_states, "count": top_5_states.index}
)

# Vytvoření sloupcového grafu počtů zemětřesení
fig = px.bar(
    earthquake_counts_df,
    x="state",
    y="count",
    title="5 států s nejvyšším počtem zemětřesení (1990-2023)",
)

# Zobrazení sloupcového grafu
fig.show()


### 3.3.2. Státy s nejnižším počtem zemětřesení



In [ ]:
# Získání 5 států s nejnižším počtem zemětřesení
bottom_5_states = (
    df.groupby("state")
    .size()
    .to_frame(name="count")
    .reset_index()
    .sort_values(by=["count"], ascending=True)
    .head(5)
)

# Vytvoření DataFrame s počty zemětřesení
bottom_earthquake_counts_df = pd.DataFrame(
    data={"state": bottom_5_states["state"],
          "count": bottom_5_states["count"]}
)

# Vytvoření sloupcového grafu počtů zemětřesení
fig = px.bar(
    bottom_earthquake_counts_df,
    x="state",
    y="count",
    title="5 států s nejnižším počtem zemětřesení (1990-2023)",
)

# Zobrazení sloupcového grafu
fig.show()


## 3.4. Analýza magnitudy

### 3.4.1. 5 nejsilnějších zemětřesení

In [ ]:
# Získání 5 nejsilnějších zemětřesení
top_5_earthquakes = (
    df.sort_values(by=["magnitudo"], ascending=False)
    .head(5)
)

# Vytvoření mapy zemětřesení
fig = px.scatter_geo(
    top_5_earthquakes,
    lat="latitude",
    lon="longitude",
    size="magnitudo",
    color="magnitudo",
    title="5 nejsilnějších zemětřesení (1990-2023)",
)

# Zobrazení mapy
fig.show()


# 4. Prediktivní modelování

## 4.1. Předzpracování dat pro modelování

Použijeme jednoduchý model lineární regrese k predikci magnitudy zemětřesení. Nejprve předzpracujeme data výběrem numerických charakteristik a rozdělením dat na trénovací a testovací.

In [ ]:
# Předzpracování dat (odstranění sloupce "place" a ponechání pouze numerických charakteristik)
numerical_columns = ["magnitudo", "depth", "latitude", "longitude"]
data_numeric = df[numerical_columns]

# Oddělení vstupních charakteristik (X) a cílové proměnné (y)
X = data_numeric.drop(columns=["magnitudo"])
y = data_numeric["magnitudo"]

# Rozdělení dat na trénovací a testovací sadu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 4.2. Trénování a vyhodnocení modelu

 Trénování modelu lineární regrese a vyhodnocení jeho výkonu pomocí střední kvadratické chyby (MSE).

In [ ]:
# Vytvoření modelu lineární regrese
model = LinearRegression()

# Trénování modelu
model.fit(X_train, y_train)

# Predikce pomocí modelu
y_pred = model.predict(X_test)

# Výpočet střední kvadratické chyby
mse = mean_squared_error(y_test, y_pred)

# Tisk střední kvadratické chyby
print("Střední kvadratická chyba:", mse)


## 4.3. Predikce modelu

Zobrazení několika příkladů predikcí z modelu

In [ ]:
y_pred = model.predict(X_test)
for i, (actual, predicted) in enumerate(zip(y_test, y_pred), 1):
    if i % 1000 == 0:
        print(f"Iterace: {i} | Skutečná magnituda: {actual:.2f} | Predikovaná magnituda: {predicted:.2f}")


# 5. Pokročilá EDA a vizualizace

 ## 5.0. Profilování dat

Generování komplexního profilu dat pomocí ydata_profiling.

In [ ]:
# Import třídy ProfileReport z knihovny ydata_profiling
from ydata_profiling import ProfileReport

# Vytvoření komplexní zprávy o profilu pro DataFrame 'df'
profile = ProfileReport(df)
profile
#profile.to_notebook_iframe()


## 5.1 Analýza Rozložení

### 5.1.1 Rozložení Magnitudy

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df["magnitudo"], bins=100, kde=True)
plt.title("Rozložení Magnitudy Zemětřesení")
plt.xlabel("Magnituda")
plt.ylabel("Četnost")

# Nastavení celých čísel na ose X
# Vytvoří značky pro celá čísla od minimální po maximální hodnotu v datech
x_ticks = range(int(df["magnitudo"].min()), int(df["magnitudo"].max()) + 1)
plt.xticks(x_ticks)

plt.show()


Rozložení magnitudy ukazuje:

*   Většina zemětřesení je mezi 0.0 a 5.0 na Richterově stupnici
*   Rozložení je mírně vychýlené doprava
*   Velmi málo zemětřesení přesahuje magnitudu 7.0


*   Pravostranná šikmost: Většina zemětřesení má nízkou magnitudu (0–2), zatímco silná zemětřesení (nad 5) jsou vzácná.

*   Bimodální rozložení: Dva vrcholy – hlavní kolem magnitudy 1 a menší kolem 4–5.

*   Četnost: Nejvyšší četnost je u magnitudy 1 (přes 300 000 výskytů).

*   Praktický význam: Slabá zemětřesení jsou běžná, silná jsou spíše vzácná.


### 5.1.2 Rozložení Hloubky


In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df["depth"], bins=100, kde=True)
plt.title("Rozložení Hloubek Zemětřesení")
plt.xlabel("Hloubka (km)")
plt.ylabel("Četnost")
plt.show()

Klíčová pozorování o hloubce:

*   Většina zemětřesení nastává v prvních 100 km hloubky
*   Existuje dlouhý chvost směrem k hlubším zemětřesením
*   Medián hloubky je přibližně 7.7 km

## 5.2 Geografická Analýza

### 5.2.1 Mapa Hustoty Zemětřesení


In [ ]:
fig = px.density_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    z='magnitudo',
    radius=10,
    center=dict(lat=0, lon=180), # Set the center
    zoom=0, # Set initial zoom
    mapbox_style="open-street-map", # Provide a style
    title="Globální Hustota Zemětřesení"
)

fig.show()

In [ ]:
fig = px.scatter_geo(
    df,
    lat='latitude',
    lon='longitude',
    size=df['magnitudo'].abs() + 0.1,  # Use absolute value and add small offset
    color='magnitudo',
    size_max=5,
    title="Globální Hustota Zemětřesení"
)

fig.show()

Geografické vzory odhalují:

*   Vysokou koncentraci podél hranic tektonických desek
*   Jasně viditelný Pacifický "Ohnivý kruh"
*   Nižší aktivitu ve vnitrozemí kontinentů

### 5.2.2 Magnituda podle Regionů


In [ ]:
top_10_states = df.groupby('state')['magnitudo'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(12,6))
top_10_states.plot(kind='bar')
plt.title("Průměrná Magnituda v Top 10 Státech/Regionech")
plt.xticks(rotation=45)
plt.show()


## 5.3 Časové Vzory


### 5.3.1 Měsíční Rozložení


In [ ]:
monthly_counts = df.groupby(df['date'].dt.month)['magnitudo'].count()
plt.figure(figsize=(12,6))
monthly_counts.plot(kind='line', marker='o')
plt.title("Měsíční Rozložení Zemětřesení")
plt.xlabel("Měsíc")
plt.ylabel("Počet Zemětřesení")
plt.show()


Sezónní vzory ukazují:

*   Mírný nárůst aktivity během zimních měsíců
*   Nižší aktivitu během léta
*   Konzistentní vzor napříč roky

### 5.3.2 Roční Trendy


In [ ]:
yearly_avg = df.groupby(df['date'].dt.year)['magnitudo'].mean()
plt.figure(figsize=(12,6))
yearly_avg.plot(kind='line', marker='o')
plt.title("Průměrná Magnituda podle Roku")
plt.xlabel("Rok")
plt.ylabel("Průměrná Magnituda")
plt.show()


Dlouhodobé trendy naznačují:

*   Relativně stabilní průměrnou magnitudu v čase
*   Mírný pokles průměrné magnitudy od roku 2010
*   Zvýšenou detekci menších zemětřesení

## 5.4 Vztahy mezi Vlastnostmi


### 5.4.1 Magnituda vs. Hloubka


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='depth', y='magnitudo', alpha=0.5)
plt.title("Magnituda vs. Hloubka")
plt.xlabel("Hloubka (km)")
plt.ylabel("Magnituda")
plt.show()


Vztah ukazuje:

*   Slabou pozitivní korelaci (r=0.357)
*   Zemětřesení s vyšší magnitudou mají tendenci vznikat ve větších hloubkách
*   Významný rozptyl ve vztahu

### 5.4.2 Analýza Výskytu Tsunami


In [ ]:
tsunami_stats = df.groupby('tsunami')['magnitudo'].agg(['mean', 'count'])


Klíčová zjištění:

*   Tsunami jsou vzácné události (0.49% zemětřesení)
*   Silná korelace mezi magnitudou a výskytem tsunami
*   Průměrná magnituda pro zemětřesení vyvolávající tsunami je výrazně vyšší

## 5.5 Statistické Shrnutí


### 5.5.1 Klíčové Metriky


### 5.5.2 Charakteristiky Rozložení


*   Rozložení magnitudy je přibližně log-normální
*   95 zemětřesení spadá mezi 0.9 a 4.3
*   Významné pravostranné zešikmení (*skewness* = 1.84)
*   Těžké chvosty rozložení (*kurtosis* = 4.12)

Tato komplexní analýza odhaluje složité vzory ve výskytu a charakteristikách zemětřesení, poskytující cenné poznatky pro vědecké porozumění i praktické hodnocení rizik.



# 6. Korelační Analýza


## 6.1 Matice Korelace Vlastností


In [ ]:
# Select only numeric columns for correlation analysis
numeric_columns = df.select_dtypes(include=np.number).columns
Corr_Matrix = df[numeric_columns].corr()

plt.figure(figsize=(25,25))
sns.heatmap(Corr_Matrix, annot=True, cmap='BuPu', center=0)
plt.title("Korelační Matice Numerických Vlastností")
plt.show()

## 6.2 Klíčové Korelace s Magnitudou

Nejvýznamnější korelace se silou zemětřesení jsou:

Nejvíce Pozitivně Korelované:

In [ ]:
print('5 Vlastností s Nejvyšší Pozitivní Korelací s Magnitudou:')
Corr_Matrix['magnitudo'].sort_values(ascending=False).head(5)


Nejvíce Negativně Korelované:



In [ ]:
print('5 Vlastností s Nejvyšší Negativní Korelací s Magnitudem:')
Corr_Matrix['magnitudo'].sort_values(ascending=True).head(5)


# 7. Klíčová Zjištění a Závěry


## 7.1 Časové Vzory


*   Frekvence zemětřesení vykazuje rostoucí trend od 1990 do 2023
*   Existují sezónní variace s mírně vyšší aktivitou v určitých měsících
*   Průměrná magnituda zůstala v průběhu času relativně stabilní

## 7.2 Geografické Rozložení


*   Kalifornie, Aljaška a Nevada vykazují nejvyšší frekvenci zemětřesení
*   Hlubší zemětřesení se vyskytují v specifických geografických oblastech
*   Pobřežní oblasti vykazují vyšší pravděpodobnost vzniku tsunami

## 7.3 Vzory Magnitudy




*   Většina zemětřesení má nízkou až střední magnitudu (M< 5.0)
*   Silná korelace mezi magnitudou a významností (r=0.939)
*   Negativní korelace mezi magnitudou a zeměpisnou šířkou (r=−0.494)

## 7.4 Výsledky Prediktivního Modelování


*   Lineární regresní model dosáhl MSE 0.899
*   Model funguje lépe pro předpovědi střední magnitudy
*   Geografické vlastnosti (zeměpisná délka, šířka) jsou silnými prediktory

# Analýza dat

In [ ]:
plt.pie(df['status'].value_counts(),labels=list(df['status'].unique()),autopct="%0.1f%%")

pie chart of current state of event(automatic or reviewed)**

As we can see reviewed state is more than automatic state

In [ ]:
df['data_type'].value_counts().sort_values(ascending=False)[:10].plot(kind='bar')
plt.show()

## Relation between magnitude and significance



In [ ]:
#The precalcualted average of magnitude is 83.02686789
#The precalculated average of significance is 1.972216904
#Here we have plot the values which are less than and eqaual to average values
y= df['magnitudo'][:10]

x = df['significance'][:10]

plt.xlabel("significance")
plt.ylabel("magnitudo")
plt.plot(x,y)

In [ ]:
sns.distplot(df['latitude'])
plt.show()

In [ ]:
sns.distplot(df['longitude'])
plt.show()

In [ ]:
state_counts = df['state'].value_counts()
print(state_counts.head(20))

In [ ]:
df

In [ ]:
# Convert the 'date' column to datetime data type
df['date'] = pd.to_datetime(df['date'])

# Assuming 'time' column is string and needs to be converted to time objects
# If 'time' is already a datetime.time object, this step is unnecessary
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S', errors='coerce').dt.time